# Data Processing

In [1]:
import csv

In [2]:
with open("password_trainv3.csv") as csv_file:
    reader = csv.reader(csv_file)
    char_ststistics = {}
    for i, row in enumerate(reader):
        if(i == 0): continue
        for c in row[1]:
            if c in char_ststistics: char_ststistics[c] += 1
            else: char_ststistics[c] = 1

In [3]:
values = []
alphapet = ''
for c in sorted(char_ststistics):
    print(c, char_ststistics[c])
    values.append(char_ststistics[c])
    alphapet += c

print(f'min value {min(values)}\nmax value {max(values)}')
print(len(alphapet))
print (alphapet)

! 66
" 72
# 114
$ 99
% 66
& 63
' 84
( 54
) 66
* 96
+ 54
, 72
- 84
. 81
/ 57
0 114
1 120
2 114
3 105
4 138
5 111
6 114
7 99
8 90
9 105
: 57
; 69
< 75
= 84
> 72
? 66
@ 60
A 54
B 78
C 84
D 75
E 84
F 69
G 84
H 54
I 87
J 78
K 39
L 72
M 69
N 78
O 75
P 75
Q 66
R 81
S 81
T 99
U 42
V 69
W 96
X 75
Y 69
Z 72
[ 60
\ 63
] 87
^ 84
_ 54
` 81
a 93
b 69
c 63
d 96
e 60
f 72
g 87
h 87
i 81
j 66
k 66
l 45
m 111
n 114
o 63
p 72
q 69
r 69
s 48
t 69
u 60
v 63
w 81
x 60
y 66
z 63
{ 57
| 63
} 81
~ 60
min value 39
max value 138
94
!"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~


In [ ]:
# balanced split


# DataSet.py

In [4]:
from torch.utils.data import Dataset, sampler, DataLoader
import torch
import torch.nn.functional as F
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt
import csv
from PIL import Image
import random

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [5]:
class Passwords_data(Dataset):
    def __init__ (self, csv_Path, imgs_path, transformers = None):
        with open(csv_Path, 'r') as csv_file:
            # images names and labels in matching indices
            reader = csv.reader(csv_file)
            self.imgs = []
            self.lables = []
            for i, row in enumerate(reader):
                if i == 0: continue
                self.imgs.append(row[0])
                self.lables.append(row[1])
                
            self.imgs_file = imgs_path
            self.transformers = transformers
    
    def __getitem__(self, index):
        # image
        img_path = self.imgs_file + '/' + self.imgs[index]
        img = Image.open(img_path).convert('L')
        #print(img.size)
        
        # image augmentation
        if self.transformers != None:
            img = self.transformers(img)
        img = transforms.ToTensor()(img)
        # lable
        lable = self.lables[index]
        
        return (img_path, img, lable)
    
    def __len__(self):
        return len(self.imgs)

In [6]:
class ResizeNormalize (object):
    def __init__(self, img_size):
        self.img_size = img_size # imgH, imgW
    
    def __call__(self, img):
        img = transforms.Resize(size=self.img_size)(transforms.ToPILImage()(img))
        img = transforms.ToTensor()(img)
        img.sub_(0.5).div_(0.5) # normalize gray scale
        return img

In [7]:
class AlignBatch(object):
    def __init__(self, imgH = 32, imgW = 100, keep_ratio = True, min_ratio = 1, padding = False):
        self.imgH = imgH
        self.imgW = imgW
        self.keep_ratio = keep_ratio
        self.min_ratio = min_ratio
        self.padding = padding
        
    def __call__(self, batch):
        img_paths, imgs, lables = zip(*batch)
        imgH = self.imgH
        imgW = self.imgW
        
        if(self.keep_ratio):
            max_ratio = 0
            for img in imgs:
                #print(img.shape)
                _, h, w = img.shape
                if max_ratio < (w/h): max_ratio = (w/h)
            if self.padding:
                imgs_padded = []
                for i, img in enumerate(imgs):
                    _, h, w = img.shape
                    w_new = h * max_ratio
                    pad = int((w_new - w) / 2)
                    #print(img.shape)
                    imgs_padded.append(F.pad(img, (pad, pad), "constant"))
                imgs = imgs_padded
            
            imgW = int(max_ratio * imgH)
            
        resizer = ResizeNormalize((imgH, imgW))
        imgs = [resizer(img).unsqueeze(0) for img in imgs]
        imgs = torch.cat(imgs, 0)
        return img_paths, imgs, lables
                    

In [8]:
class MatchingSampler(sampler.Sampler):
    
    def __init__ (self, data_source, batch_size):
        self.data_source = data_source
        self.number_sampels = len(data_source)
        self.batch_size = batch_size
        
    def __iter__ (self):
        n_batches = len(self)//self.batch_size
        tail = len(self) % self.batch_size
        index = torch.LongTensor(len(self)).fill_(0)
        for i in range(n_batches):
            random_start = random.randint(0, len(self)-self.batch_size)
            b_indx = random_start + torch.arange(0, self.batch_size)
            index[i*self.batch_size : (i+1) * self.batch_size] = b_indx
        if tail:
            random_start = random.randint(0, len(self)-tail)
            b_indx = random_start + torch.arange(0, tail-1)
            index[n_batches * self.batch_size:] = b_indx
            
        return iter(index)
        
    def __len__(self):
        return self.number_sampels

In [ ]:
dataset = Passwords_data('password_trainv3.csv', 'train_passwordv3')

sampler = MatchingSampler(dataset, 4)
align = AlignBatch()
dataloder = DataLoader(dataset, batch_size=4, collate_fn=align, shuffle= False)

img_pathes, imgs, lables = next(iter(dataloder))

for sample in zip(img_pathes, imgs, lables):
    #print(sample)
    img_path, img, lable = sample
    print(img.shape)
    plt.imshow(img.squeeze(0).numpy(), cmap='gray')
    plt.show()
    print(lable)

# Model

In [9]:
import torch.nn as nn

In [10]:
class BiDrictionalLSTM(nn.Module):
    def __init__(self, nIn, nHidden, nOut):
        super(BiDrictionalLSTM, self).__init__()
        self.rnn = nn.LSTM(nIn, nHidden, bidirectional=True)
        self.embedding = nn.Linear(nHidden * 2, nOut)
    
    def forward(self, input):
        recurrent, _ = self.rnn(input)
        t_steps, b_size, h_num = recurrent.shape
        recurrent = recurrent.view(t_steps*b_size, h_num) # prepare the linear layer input
        
        output = self.embedding(recurrent)
        output = output.view(t_steps, b_size, -1)
        
        return output

In [ ]:
'''
class ConvLayer(nn.Module):
    def __init__(self, inFilt, outFilt, kerSiz, padSiz, strideSiz, b_n = False):
        super(ConvLayer, self).__init__()
        self.conv = nn.Conv2d(inFilt, outFilt, kerSiz, padSiz, strideSiz)
        self.relu = nn.ReLU(inplace=True)
    
    def
'''

In [11]:
class CRNN(nn.Module):
    def __init__(self, imgH, inChannal, nClasses, nHidden, nLSTMs = 2):
        super(CRNN, self).__init__()
        assert imgH == 32 , 'the image input hight must be 32'
        
        ks = [3, 3, 3, 3, 3, 3, 2] # kernal Size
        ps = [1, 1, 1, 1, 1, 1, 0] # padding
        ss = [1, 1, 1, 1, 1, 1, 1] # stride
        fn = [64, 128, 256, 256, 512, 512, 512] # filters number
        
        cnn = nn.Sequential()
        
        
        def conv_layer(layNum, b_n = False):
            nIn = inChannal if layNum == 0 else fn[layNum-1]
            nOut = fn[layNum]
            # Conv Layer
            cnn.add_module(f'conv{layNum}', nn.Conv2d(nIn, nOut, ks[layNum], ss[layNum], ps[layNum]))
            # btach normalization
            if b_n:
                cnn.add_module(f'batchnorm{layNum}', nn.BatchNorm2d(nOut))
            # non Linearity (ReLU)
            cnn.add_module(f'relu{layNum}', nn.ReLU(inplace=True))
            
        # Cnn Arch
        conv_layer(0)
        cnn.add_module(f'pooling{0}', nn.MaxPool2d(2, 2))  # 64 x 16
        conv_layer(1)
        cnn.add_module(f'pooling{1}', nn.MaxPool2d(2, 2))  # 128 x 8
        conv_layer(2, b_n=True)
        conv_layer(3)
        # the irregular shape of stride and padding beacause of the shape of some char like (i, ..)
        cnn.add_module(f'pooling{2}', nn.MaxPool2d((2, 2), (2, 1), (0, 1))) # 256 x 4
        conv_layer(4, b_n=True)
        conv_layer(5)
        cnn.add_module(f'pooling{3}', nn.MaxPool2d((2, 2), (2, 1), (0, 1))) # 512 x 2
        conv_layer(6, b_n=True) #  512 x 1

        self.cnn = cnn

        # Rnn Arch
        rnn = nn.Sequential(
            BiDrictionalLSTM(512, nHidden, nHidden),
            BiDrictionalLSTM(nHidden, nHidden, nClasses)
        )

        self.rnn = rnn
        
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input):
        # cnn pass
        conv = self.cnn(input)
        b, c, h, w = conv.shape
        
        assert h == 1, 'the hight after cnn must equal 1'
        
        conv = conv.squeeze(2)
        conv = conv.permute(2, 0, 1) # sequance, batch, features
        
        # rnn pass 
        rnn = self.rnn(conv)
        
        output = self.softmax(rnn)
        
        return output


In [ ]:
crnn = CRNN(32, 1, 37, 256)

In [ ]:
crnn.load_state_dict(torch.load('crnn.pth'))

In [ ]:
# model state dict
st = crnn.state_dict()
for name in st:
    print(name)

In [ ]:
# checkpoint state dic
st = torch.load('crnn.pth')
for name,k in st.items():
    print(name)

# Utilties

## Logger

In [12]:
import tensorflow as tf


class Logger(object):
    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        self.writer = tf.summary.FileWriter(log_dir)

    def scalar_summary(self, tag, value, step):
        """Log a scalar variable."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)

    def list_of_scalars_summary(self, tag_value_pairs, step):
        """Log scalar variables."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value) for tag, value in tag_value_pairs])
        self.writer.add_summary(summary, step)

## Label String Convertion

In [13]:
class strLabelConverter(object):
    """Convert between str and label.

    NOTE:
        Insert `blank` to the alphabet for CTC.

    Args:
        alphabet (str): set of the possible characters.
        ignore_case (bool, default=True): whether or not to ignore all of the case.
    """

    def __init__(self, alphabet, ignore_case=True):
        self._ignore_case = ignore_case
        if self._ignore_case:
            alphabet = alphabet.lower()
        self.alphabet = alphabet + '-'  # for `-1` index

        self.dict = {}
        for i, char in enumerate(alphabet):
            # NOTE: 0 is reserved for 'blank' required by wrap_ctc
            self.dict[char] = i + 1

    def encode(self, text):
        """Support batch or single str.

        Args:
            text (str or list of str): texts to convert.

        Returns:
            torch.IntTensor [length_0 + length_1 + ... length_{n - 1}]: encoded texts.
            torch.IntTensor [n]: length of each text.
        """
        if isinstance(text, str):
            text = [
                self.dict[char.lower() if self._ignore_case else char]
                for char in text
            ]
            length = [len(text)]
        elif isinstance(text, collections.abc.Iterable):
            length = [len(s) for s in text]
            text = ''.join(text)
            text, _ = self.encode(text)
        return (torch.IntTensor(text), torch.IntTensor(length))

    def decode(self, t, length, raw=False):
        """Decode encoded texts back into strs.

        Args:
            torch.IntTensor [length_0 + length_1 + ... length_{n - 1}]: encoded texts.
            torch.IntTensor [n]: length of each text.

        Raises:
            AssertionError: when the texts and its length does not match.

        Returns:
            text (str or list of str): texts to convert.
        """
        if length.numel() == 1:
            length = length[0]
            assert t.numel() == length, "text with length: {} does not match declared length: {}".format(t.numel(), length)
            if raw:
                return ''.join([self.alphabet[i - 1] for i in t]), [i - 1 for i in t]
            else:
                char_list = []
                lables_list = []
                for i in range(length):
                    if t[i] != 0 and (not (i > 0 and t[i - 1] == t[i])):
                        char_list.append(self.alphabet[t[i] - 1])
                        lables_list.append(t[i] - 1)
                return ''.join(char_list), lables_list
        else:
            # batch mode
            assert t.numel() == length.sum(), "texts with length: {} does not match declared length: {}".format(t.numel(), length.sum())
            texts = []
            lables = []
            index = 0
            for i in range(length.numel()):
                l = length[i]
                text, lable = self.decode(
                        t[index:index + l], torch.IntTensor([l]), raw=raw)
                texts.append(text)
                lables.append(lable)
                index += l
            return texts, lables

# Training

In [14]:
import torch 
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
import PIL
import time
import collections

import os

In [15]:
os.makedirs('checkpoints', exist_ok=True)

## Test Tranforms

In [ ]:
train_transforms = transforms.Compose([transforms.ColorJitter(brightness=.5, contrast=.5, saturation=.5)])

In [ ]:
train_transforms = transforms.Compose([transforms.RandomAffine(degrees=0, translate=(.03,.03))])

In [ ]:
train_transforms = transforms.Compose([transforms.RandomAffine(degrees=0, scale=(.95,1.05))])

In [ ]:
train_transforms = transforms.Compose([transforms.RandomAffine(degrees=0, shear=20)])

In [ ]:
train_transforms = transforms.Compose([transforms.RandomRotation(degrees=3, expand=True), 
                                       transforms.ColorJitter(brightness=.5, contrast=.5, saturation=.5)])

In [ ]:
train_transforms = transforms.Compose([transforms.RandomPerspective()])

In [ ]:
dataset = Passwords_data('password_trainv3.csv', 'train_passwordv3', transformers=train_transforms)

sampler = MatchingSampler(dataset, 4)
align = AlignBatch()
dataloder = DataLoader(dataset, batch_size=4, collate_fn=align, shuffle= True)

img_pathes, imgs, lables = next(iter(dataloder))
#print(converter.encode(lables))

for sample in zip(img_pathes, imgs, lables):
    #print(sample)
    img_path, img, lable = sample
    print(img.shape)
    plt.imshow(img.squeeze(0).numpy(), cmap='gray')
    plt.show()
    print(lable)

## continue training

In [16]:
# training variables 
epochs_num = 200
batch_size = 16
cuda = torch.cuda.is_available()
n_workers = 4
nClasses = len(char_ststistics) + 1
inChannels = 1
imgH = 32
nHidden = 256
lr = .001
test_display = 4
val_each = 1
use_pretrained = True
pre_trained = 'crnn.pth'
fixed_seed = 12
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'detect cuda device? {cuda}')
print(f'number of classes id {nClasses-1} + blank')

detect cuda device? True
number of classes id 94 + blank


In [17]:
# fix seeds
random.seed(fixed_seed)
np.random.seed(fixed_seed)
torch.manual_seed(fixed_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [18]:
train_transforms = transforms.Compose([
    transforms.RandomApply([
        transforms.RandomChoice([
            transforms.RandomAffine(degrees=0, translate=(.03,.03)),
            transforms.RandomAffine(degrees=0, scale=(.95,1.05)),
            transforms.RandomAffine(degrees=0, shear=20),
            transforms.RandomRotation(degrees=3, expand=True)]),
        transforms.ColorJitter(brightness=.3, contrast=.3, saturation=.3)],  p=0.5)])

In [19]:
train_dataset = Passwords_data('password_train.csv', 'train_passwordv3', transformers=train_transforms)

#sampler = MatchingSampler(dataset, batch_size)
align = AlignBatch()
train_dataloder = DataLoader(train_dataset, batch_size=batch_size, collate_fn=align, 
                       shuffle= True, num_workers=n_workers)

In [20]:
val_dataset = Passwords_data('password_val.csv', 'train_passwordv3')

#sampler = MatchingSampler(dataset, batch_size)
align = AlignBatch()
val_dataloder = DataLoader(val_dataset, batch_size=batch_size, collate_fn=align, 
                       shuffle= True, num_workers=n_workers)

In [21]:
converter = strLabelConverter(alphapet, ignore_case=False)
criterion = nn.CTCLoss()

In [22]:
def weights_init(m):
    class_name = m.__class__.__name__
    if class_name.find('conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif class_name.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [23]:
crnn = CRNN(imgH, inChannels, nClasses, nHidden)
crnn.apply(weights_init)
if use_pretrained :
    model_dict = crnn.state_dict() # state of the current model
    pretrained_dict = torch.load(pre_trained) # state of the pretrained model
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k != 'rnn.1.embedding.weight' and k != 'rnn.1.embedding.bias'} # remove the classifier from the state
    classifier_dict = {k: v for k, v in model_dict.items() if k == 'rnn.1.embedding.weight' or k == 'rnn.1.embedding.bias'} # get the classifier weight from new model
    pretrained_dict.update(classifier_dict) # update without classifier
    crnn.load_state_dict(pretrained_dict)
    
print(crnn)

CRNN(
  (cnn): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu0): ReLU(inplace)
    (pooling0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace)
    (pooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace)
    (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace)
    (pooling2): MaxPool2d(kernel_size=(2, 2), stride=(2, 1), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, trac

In [24]:
if cuda :
    crnn = crnn.to(device)
    criterion = criterion.to(device)

In [25]:
optimizer = optim.Adam(crnn.parameters(), lr=lr)
lr_sheduler = optim.lr_scheduler.StepLR(optimizer, 100, gamma=.5)

In [26]:
# define logger file
os.makedirs('logs', exist_ok=True)
logger = Logger('logs')

In [27]:
def train(model, criterion, optimizer, logger, train_dataloder, batch_size, epoch_num):
    model.train()
    epoch_loss = 0
    samples_num = 0
    
    for batch_i, (_, imgs, targets) in enumerate(train_dataloder):
        batches_done = len(train_dataloder) * epoch_num + batch_i
        samples_num += imgs.shape[0]
        
        # move to device and create variables
        imgs = Variable(imgs.to(device))
        targets, lenghts = converter.encode(targets)
        targets = Variable(targets.to(device), requires_grad=False)
        t_lens = Variable(lenghts, requires_grad=False)
        
        # pass to the network
        preds = model(imgs)
        preds_size = Variable(torch.IntTensor([preds.shape[0]] * imgs.shape[0]))
        
        # loss
        #print(preds_size.shape)
        loss = criterion(preds, targets.cpu(), preds_size, t_lens)
        epoch_loss += loss * imgs.shape[0]
        logger.scalar_summary('loss_batches', loss, batches_done)
        print(f'Epoch {epoch_num}, Batch {batch_i}/{len(train_dataloder)} : Loss = {loss}')
        
        # optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    epoch_loss /= samples_num
    logger.scalar_summary('loss_epochs', epoch_loss, epoch_num)
    
    return epoch_loss

In [28]:
def val(model, criterion, logger, val_dataloder, epoch_num, batch_size=16):
    model.eval()
    
    nCorrect_words = 0
    #nCorrect_chars = 0
    val_loss = 0
    samples_num = 0
    
    for batch_i, (_, imgs, targets) in enumerate(val_dataloder):
        samples_num += imgs.shape[0]
        
        # move to device and create variables
        imgs = Variable(imgs.to(device), requires_grad=False)
        targets_encoded, lenghts = converter.encode(targets)
        targets_encoded = Variable(targets_encoded.to(device), requires_grad=False)
        t_lens = Variable(lenghts, requires_grad=False)
        
        with torch.no_grad():
            # pass to the network
            preds = model(imgs)
            preds_size = Variable(torch.IntTensor([preds.shape[0]] * imgs.shape[0]))

            # loss
            loss = criterion(preds, targets_encoded.cpu(), preds_size, t_lens)
            val_loss += loss * imgs.shape[0]
            
            # get the nework prediction
            _, preds = preds.max(2)
            preds = preds.transpose(1,0).contiguous().view(-1)
            words_preds, lables_preds = converter.decode(preds, preds_size)
            
            for word_pred, target in zip(words_preds, targets):
                if word_pred == target:
                    nCorrect_words += 1
    
    # display some of the network prediction
    row_preds, _ = converter.decode(preds, preds_size, raw=True)[:test_display]

    for row_pred, word_pred, gt in zip(row_preds, words_preds, targets):
        print(f'{row_pred} => {word_pred}, Ground Truth is {gt}')
    
    #compute loss and accurcy
    word_accurcy = nCorrect_words / samples_num
    val_loss /= samples_num
    logger.scalar_summary('val_loss', val_loss, epoch_num)
    logger.scalar_summary('val_WordAccurcy', word_accurcy, epoch_num)
    
    return val_loss, word_accurcy

In [29]:
best_acc = 0
for epoch in range(epochs_num):
    
    tick = time.time()
    train_loss = train(crnn, criterion, optimizer, logger, 
                       train_dataloder, batch_size, epoch)
    tock = time.time()
    
    print(f'Epoch {epoch} finished in {(tock - tick) / 60} minutes')
    print(f'Epoch {epoch} training_loss = {train_loss}')
    
    if epoch % val_each == 0:
        val_loss, val_accurcy = val(crnn, criterion, logger, val_dataloder,
                                    epoch, batch_size)
        print(f'Epoch {epoch} val_loss = {val_loss}, word_accuracy = {val_accurcy}')
            
        # save best checkpoint
        if best_acc <= val_accurcy:
            best_acc = val_accurcy
            checkpoint = {'input_hight':32,
                          'output_size':len(alphapet)+1,
                          'alphapet':alphapet,
                          'train_transforms':train_transforms,
                          'optim_dic':optimizer.state_dict(),
                          'state_dic':crnn.state_dict(),
                          'epoch':epoch
                         }
            torch.save(checkpoint,'checkpoints/best_checkpoint.pth')
    
    # save last epoch
    checkpoint = {'input_hight':32,
                  'output_size':len(alphapet)+1,
                  'alphapet':alphapet,
                  'train_transforms':train_transforms,
                  'optim_dic':optimizer.state_dict(),
                  'state_dic':crnn.state_dict(),
                  'epoch':epoch
                 }
    torch.save(checkpoint,'checkpoints/last_checkpoint.pth')
    
    lr_sheduler.step()
    
print(f'the best accurcay is {best_acc}')
            

Epoch 0, Batch 0/32 : Loss = 21.643980026245117
Epoch 0, Batch 1/32 : Loss = 19.478717803955078
Epoch 0, Batch 2/32 : Loss = 23.58216094970703
Epoch 0, Batch 3/32 : Loss = 21.24152946472168
Epoch 0, Batch 4/32 : Loss = 19.950885772705078
Epoch 0, Batch 5/32 : Loss = 18.413835525512695
Epoch 0, Batch 6/32 : Loss = 16.70687484741211
Epoch 0, Batch 7/32 : Loss = 14.276725769042969
Epoch 0, Batch 8/32 : Loss = 12.767298698425293
Epoch 0, Batch 9/32 : Loss = 10.44369888305664
Epoch 0, Batch 10/32 : Loss = 8.966351509094238
Epoch 0, Batch 11/32 : Loss = 6.584331035614014
Epoch 0, Batch 12/32 : Loss = 5.221884727478027
Epoch 0, Batch 13/32 : Loss = 4.7490434646606445
Epoch 0, Batch 14/32 : Loss = 4.781960487365723
Epoch 0, Batch 15/32 : Loss = 5.086452484130859
Epoch 0, Batch 16/32 : Loss = 5.437365531921387
Epoch 0, Batch 17/32 : Loss = 5.5425124168396
Epoch 0, Batch 18/32 : Loss = 5.738100528717041
Epoch 0, Batch 19/32 : Loss = 5.7718353271484375
Epoch 0, Batch 20/32 : Loss = 5.766099929809

Epoch 4, Batch 0/32 : Loss = 2.3384246826171875
Epoch 4, Batch 1/32 : Loss = 2.160064935684204
Epoch 4, Batch 2/32 : Loss = 1.947206974029541
Epoch 4, Batch 3/32 : Loss = 2.078225612640381
Epoch 4, Batch 4/32 : Loss = 2.355708599090576
Epoch 4, Batch 5/32 : Loss = 1.9781115055084229
Epoch 4, Batch 6/32 : Loss = 2.270360231399536
Epoch 4, Batch 7/32 : Loss = 1.926094651222229
Epoch 4, Batch 8/32 : Loss = 1.7926443815231323
Epoch 4, Batch 9/32 : Loss = 1.9302606582641602
Epoch 4, Batch 10/32 : Loss = 1.7675186395645142
Epoch 4, Batch 11/32 : Loss = 1.7046866416931152
Epoch 4, Batch 12/32 : Loss = 2.185209274291992
Epoch 4, Batch 13/32 : Loss = 1.6642831563949585
Epoch 4, Batch 14/32 : Loss = 1.577927589416504
Epoch 4, Batch 15/32 : Loss = 1.8931968212127686
Epoch 4, Batch 16/32 : Loss = 1.7782456874847412
Epoch 4, Batch 17/32 : Loss = 1.4123470783233643
Epoch 4, Batch 18/32 : Loss = 1.352522611618042
Epoch 4, Batch 19/32 : Loss = 1.9937174320220947
Epoch 4, Batch 20/32 : Loss = 1.3488634

Epoch 8, Batch 0/32 : Loss = 0.11571566015481949
Epoch 8, Batch 1/32 : Loss = 0.3791310787200928
Epoch 8, Batch 2/32 : Loss = 0.13696901500225067
Epoch 8, Batch 3/32 : Loss = 0.14420795440673828
Epoch 8, Batch 4/32 : Loss = 0.10432037711143494
Epoch 8, Batch 5/32 : Loss = 0.24922938644886017
Epoch 8, Batch 6/32 : Loss = 0.09343170374631882
Epoch 8, Batch 7/32 : Loss = 0.3146168887615204
Epoch 8, Batch 8/32 : Loss = 0.365764319896698
Epoch 8, Batch 9/32 : Loss = 0.10468517243862152
Epoch 8, Batch 10/32 : Loss = 0.26260876655578613
Epoch 8, Batch 11/32 : Loss = 0.1904647946357727
Epoch 8, Batch 12/32 : Loss = 0.12089686095714569
Epoch 8, Batch 13/32 : Loss = 0.11706064641475677
Epoch 8, Batch 14/32 : Loss = 0.3630850613117218
Epoch 8, Batch 15/32 : Loss = 0.27358824014663696
Epoch 8, Batch 16/32 : Loss = 0.16415202617645264
Epoch 8, Batch 17/32 : Loss = 0.10589954257011414
Epoch 8, Batch 18/32 : Loss = 0.11847132444381714
Epoch 8, Batch 19/32 : Loss = 0.1368398368358612
Epoch 8, Batch 20

------k----$-----l--F-----DD-----e----hh----]---k---0-----\----X------ => k$lFDeh]k0\X, Ground Truth is k$IFDeh]k0\X
-----=------0------[---X----))---RR-----88----ii--PP-----w-------)---- => =0[X)R8iPw), Ground Truth is Err:509
----X----7---00---j---@-----S----Z----L---44----C----m-------MM------- => X70j@SZL4CmM, Ground Truth is X70j@SZL4CmM
-;-------3------\\---$$------>------SS-----\\-----M-------ii---B------ => ;-3\$>S\MiB, Ground Truth is :-3\$>S\MiB
Epoch 11 val_loss = 0.5100294947624207, word_accuracy = 0.56
Epoch 12, Batch 0/32 : Loss = 0.09622953832149506
Epoch 12, Batch 1/32 : Loss = 0.17512357234954834
Epoch 12, Batch 2/32 : Loss = 0.03868137300014496
Epoch 12, Batch 3/32 : Loss = 0.07819114625453949
Epoch 12, Batch 4/32 : Loss = 0.04810584336519241
Epoch 12, Batch 5/32 : Loss = 0.07015082985162735
Epoch 12, Batch 6/32 : Loss = 0.03713603317737579
Epoch 12, Batch 7/32 : Loss = 0.1520155519247055
Epoch 12, Batch 8/32 : Loss = 0.0985426977276802
Epoch 12, Batch 9/32 : Loss = 0

Epoch 15, Batch 21/32 : Loss = 0.030972333624958992
Epoch 15, Batch 22/32 : Loss = 0.04172181338071823
Epoch 15, Batch 23/32 : Loss = 0.034759312868118286
Epoch 15, Batch 24/32 : Loss = 0.08018998801708221
Epoch 15, Batch 25/32 : Loss = 0.04066680371761322
Epoch 15, Batch 26/32 : Loss = 0.03049156442284584
Epoch 15, Batch 27/32 : Loss = 0.01913805678486824
Epoch 15, Batch 28/32 : Loss = 0.05824131891131401
Epoch 15, Batch 29/32 : Loss = 0.1870623081922531
Epoch 15, Batch 30/32 : Loss = 0.01550145260989666
Epoch 15, Batch 31/32 : Loss = 0.018260706216096878
Epoch 15 finished in 0.05164132912953694 minutes
Epoch 15 training_loss = 0.06070663407444954
-----Y----W-----]--i--\--|----MM----<-----MM----8---8----- => YW]i\|M<M88, Ground Truth is YW]i\|M<M88
---k----O-----/--,--yy--cc---*----1---PP---}---#----B----- => kO/,yc*1P}#B, Ground Truth is kO/,yc*1P}#B
--BB---.---Y---I--W-------6----F----h----X---'--Y----22--- => B.YIW6FhX'Y2, Ground Truth is B.YIW6FhX'Y2
-----kk--$$---l--F----D----e--

...

Epoch 183, Batch 6/32 : Loss = 0.00021070113871246576
Epoch 183, Batch 7/32 : Loss = 0.0005928511382080615
Epoch 183, Batch 8/32 : Loss = 0.00022346689365804195
Epoch 183, Batch 9/32 : Loss = 0.00016888888785615563
Epoch 183, Batch 10/32 : Loss = 0.00029671992524527013
Epoch 183, Batch 11/32 : Loss = 0.0002234206476714462
Epoch 183, Batch 12/32 : Loss = 0.0004078174533788115
Epoch 183, Batch 13/32 : Loss = 0.0003533453564159572
Epoch 183, Batch 14/32 : Loss = 0.00038459745701402426
Epoch 183, Batch 15/32 : Loss = 0.00036257179453969
Epoch 183, Batch 16/32 : Loss = 0.0001893438893603161
Epoch 183, Batch 17/32 : Loss = 0.00025383499450981617
Epoch 183, Batch 18/32 : Loss = 0.00015636382158845663
Epoch 183, Batch 19/32 : Loss = 0.0001728383358567953
Epoch 183, Batch 20/32 : Loss = 0.03645177558064461
Epoch 183, Batch 21/32 : Loss = 0.0007539151702076197
Epoch 183, Batch 22/32 : Loss = 0.0002381213998887688
Epoch 183, Batch 23/32 : Loss = 0.00021102032042108476
Epoch 183, Batch 24/32 : Los

----E-----00----[---x----)---RR----8----i---P-----w------)-- => E-0[x)R8iPw), Ground Truth is Err:509
---JJ--;--q----++---/---z---y---U-------%----U-----1---x---- => J;q+/zyU%U1x, Ground Truth is J;q+/zyU%U1x_
---0-----J----!--(--;---A-----33--,,--'--)---r---r---7------ => 0J!(;A3,')rr7, Ground Truth is 0J!(;A3,')rr7
---7---n----DD----P----n---t--w-----d----\--QQ----a----RR--- => 7nDPntwd\QaR, Ground Truth is 7nDPntwd\QaR
Epoch 186 val_loss = 0.7759115695953369, word_accuracy = 0.67
Epoch 187, Batch 0/32 : Loss = 0.016662804409861565
Epoch 187, Batch 1/32 : Loss = 0.0015322251711040735
Epoch 187, Batch 2/32 : Loss = 0.012344330549240112
Epoch 187, Batch 3/32 : Loss = 0.0025554527528584003
Epoch 187, Batch 4/32 : Loss = 0.00032707356149330735
Epoch 187, Batch 5/32 : Loss = 0.0045097218826413155
Epoch 187, Batch 6/32 : Loss = 0.0003692147438414395
Epoch 187, Batch 7/32 : Loss = 0.0007314280373975635
Epoch 187, Batch 8/32 : Loss = 0.0006781365373171866
Epoch 187, Batch 9/32 : Loss = 0.000

Epoch 190, Batch 15/32 : Loss = 0.004518902860581875
Epoch 190, Batch 16/32 : Loss = 0.0004195378569420427
Epoch 190, Batch 17/32 : Loss = 0.00017721098265610635
Epoch 190, Batch 18/32 : Loss = 0.0006555357249453664
Epoch 190, Batch 19/32 : Loss = 0.0003509953967295587
Epoch 190, Batch 20/32 : Loss = 0.00017467368161305785
Epoch 190, Batch 21/32 : Loss = 0.00019261080888099968
Epoch 190, Batch 22/32 : Loss = 0.030117014423012733
Epoch 190, Batch 23/32 : Loss = 0.0006931365933269262
Epoch 190, Batch 24/32 : Loss = 0.0002352692245040089
Epoch 190, Batch 25/32 : Loss = 0.00026854508905671537
Epoch 190, Batch 26/32 : Loss = 0.001300775445997715
Epoch 190, Batch 27/32 : Loss = 0.004301915410906076
Epoch 190, Batch 28/32 : Loss = 0.0003382473369129002
Epoch 190, Batch 29/32 : Loss = 0.00017427149577997625
Epoch 190, Batch 30/32 : Loss = 0.0006648949347436428
Epoch 190, Batch 31/32 : Loss = 0.25436651706695557
Epoch 190 finished in 0.0508025328318278 minutes
Epoch 190 training_loss = 0.003105

Epoch 194, Batch 0/32 : Loss = 0.0007660623523406684
Epoch 194, Batch 1/32 : Loss = 0.00036902871215716004
Epoch 194, Batch 2/32 : Loss = 0.00040545265073888004
Epoch 194, Batch 3/32 : Loss = 0.000284641602775082
Epoch 194, Batch 4/32 : Loss = 0.0036837151274085045
Epoch 194, Batch 5/32 : Loss = 0.0009622670477256179
Epoch 194, Batch 6/32 : Loss = 0.00019137037452310324
Epoch 194, Batch 7/32 : Loss = 0.0001694269012659788
Epoch 194, Batch 8/32 : Loss = 0.0011640394804999232
Epoch 194, Batch 9/32 : Loss = 0.0006228981073945761
Epoch 194, Batch 10/32 : Loss = 0.0002715873997658491
Epoch 194, Batch 11/32 : Loss = 0.00042630391544662416
Epoch 194, Batch 12/32 : Loss = 0.00022116115724202245
Epoch 194, Batch 13/32 : Loss = 0.0002954297815449536
Epoch 194, Batch 14/32 : Loss = 0.00036073673982173204
Epoch 194, Batch 15/32 : Loss = 0.0003707194118760526
Epoch 194, Batch 16/32 : Loss = 0.00024115503765642643
Epoch 194, Batch 17/32 : Loss = 0.00034064322244375944
Epoch 194, Batch 18/32 : Loss =

Epoch 197, Batch 24/32 : Loss = 0.00014534805086441338
Epoch 197, Batch 25/32 : Loss = 0.0007714082021266222
Epoch 197, Batch 26/32 : Loss = 0.0007900800555944443
Epoch 197, Batch 27/32 : Loss = 0.0001483802916482091
Epoch 197, Batch 28/32 : Loss = 0.0006772043998353183
Epoch 197, Batch 29/32 : Loss = 0.00029165527666918933
Epoch 197, Batch 30/32 : Loss = 0.0005848376313224435
Epoch 197, Batch 31/32 : Loss = 0.00023684208281338215
Epoch 197 finished in 0.05203169584274292 minutes
Epoch 197 training_loss = 0.00044769348460249603
---d----:---X----9----e-----a----F--,,-88-------VV----R------ => d:X9eaF,8-VR, Ground Truth is d:X9eaF,8-VR
----/----MM-----o----EE---^---3----x---/---&----66----X------ => /MoE^3x/&6X, Ground Truth is /MoE^3x/&6X
---#-----G----9----E---I--=---hh---5---#----2----J---)--k---- => #G9EI=h5#2J)k, Ground Truth is #G9EI=h5#2J)k
-:------33----\\---$$---->------S-----\-----MM----ii---B----- => :-3\$>S\MiB, Ground Truth is :-3\$>S\MiB
Epoch 197 val_loss = 0.8271462917327

# Model Eval

In [ ]:
import sklearn.metrics as metrics

In [ ]:
# F1 score
def F1_score_comp(model, criterion, val_dataloder, epoch_num, batch_size=16):
    model.eval()
    
    nCorrect_words = 0
    #nCorrect_chars = 0
    val_loss = 0
    samples_num = 0
    y_pred = []
    y_targets = []
    worng_samples = []
    worng_preds = []
    
    for batch_i, (img_path, imgs, targets) in enumerate(val_dataloder):
        samples_num += imgs.shape[0]
        
        # move to device and create variables
        imgs = Variable(imgs.to(device), requires_grad=False)
        targets_encoded, lenghts = converter.encode(targets)
        targets_encoded = Variable(targets_encoded.to(device), requires_grad=False)
        t_lens = Variable(lenghts, requires_grad=False)
        
        with torch.no_grad():
            # pass to the network
            preds = model(imgs)
            preds_size = Variable(torch.IntTensor([preds.shape[0]] * imgs.shape[0]))

            # loss
            loss = criterion(preds, targets_encoded.cpu(), preds_size, t_lens)
            val_loss += loss * imgs.shape[0]
            
            # get the nework prediction
            _, preds = preds.max(2)
            preds = preds.transpose(1,0).contiguous().view(-1)
            words_preds, lables_preds = converter.decode(preds.cpu(), preds_size)
            
            for i, (word_pred, target) in enumerate(zip(words_preds, targets)):
                if word_pred == target:
                    nCorrect_words += 1
                else:
                    worng_samples.append(img_path[i])
                    worng_preds.append(word_pred)
                    
                # char level evalutaion
                target_lable = [converter.dict[c] for c in target]
                pred_lable = [converter.dict[c] for c in word_pred]
                if len(target_lable) > len(pred_lable):
                    pred_lable.extend([len(alphapet)]*abs(len(target_lable) - len(pred_lable)))
                elif len(target_lable) < len(pred_lable):
                    target_lable.extend([len(alphapet)]*abs(len(target_lable) - len(pred_lable)))
                assert len(pred_lable) == len(target_lable), f'not matched{len(target_lable)}, {len(pred_lable)}'
                y_pred.extend(pred_lable)
                y_targets.extend(target_lable)
                for c_p, c_t in zip(word_pred, target):
                    if(c_t == ''and c_p=='m'):
                        print(img_path[i])
                
    
    # display some of the network prediction
    row_preds, _ = converter.decode(preds, preds_size, raw=True)[:test_display]

    for row_pred, word_pred, gt in zip(row_preds, words_preds, targets):
        print(f'{row_pred} => {word_pred}, Ground Truth is {gt}')
    
    #compute loss and accurcy
    word_accurcy = nCorrect_words / samples_num
    val_loss /= samples_num
    
    char_acc = 0
    for c_p, c_t in zip(y_pred, y_targets):
        if(c_p == c_t): char_acc += 1
    char_acc /= len(y_pred)
    
    prec = metrics.precision_score(y_targets, y_pred, average='macro')
    recall = metrics.recall_score(y_targets, y_pred, average='macro')
    f1_score = metrics.f1_score(y_targets, y_pred, average='macro')
    
    return val_loss, word_accurcy, char_acc, prec, recall, f1_score, y_pred, y_targets, worng_samples, worng_preds

In [ ]:
val_loss, word_accurcy, char_acc, prec, recall, f1_score, y_pred, y_targets, wrong_smaples, wrong_pred = F1_score_comp(crnn, criterion, val_dataloder,
                                    epoch, batch_size)

print(f'loss: {val_loss}, word_acc: {word_accurcy}, char_acc: {char_acc}, \
prec: {prec}, recall: {recall}, f1_score: {f1_score}')

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

# plot confusion matrix
confusion_matrix = np.zeros((len(alphapet)+1, len(alphapet)+1))
for c_p, c_t in zip(y_pred, y_targets):
    confusion_matrix[c_t, c_p] += 1
    
df = pd.DataFrame(confusion_matrix, index=[c for c in (alphapet+'-')], columns=[c for c in (alphapet+'-')])
plt.figure(figsize=(30,30))
sn.set(font_scale=1)
sn.heatmap(df, annot=True)

In [ ]:
# display wrong samples\
for path, pred in zip(wrong_smaples, wrong_pred):
    print(pred)
    plt.imshow(np.array(Image.open(path).convert('L')), cmap='gray')
    plt.show()

In [ ]:
len(wrong_pred)

In [ ]:
print(confusion_matrix[converter.dict[''], converter.dict['y']])

In [ ]:
np.sum(confusion_matrix)

In [ ]:
len(y_pred)